# Hospitals

#### Load python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import numpy as np

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

#### Read data

In [4]:
states = pd.read_csv(
    "https://raw.githubusercontent.com/stiles/notebooks/5a9a1a8395c7924abe6c7392fdabd58fda8d8d8d/census/raw/ap_states.csv"
)

In [5]:
states["name"] = states["name"].str.upper()

In [6]:
today = pd.to_datetime("today").strftime("%-m-%-d-%y")

---

In [7]:
url = "https://69c46ad6-7f7a-4e40-9f99-cdf68e8848bf.filesusr.com/ugd/a53784_d422b73592324d48ac944b66ef52fd1c.csv?dn=Raw%20Data%203-2-22.csv"

In [8]:
src = pd.read_csv(url)

In [9]:
src.columns = src.columns.str.lower().str.replace(" ", "_", regex=False)

In [10]:
src.head()

,submission_time,professional_title,level_of_care,normal_ratio,surged_ratio,how_often_do_you_surge?,which_shift?,facility_name,city,state,comments,do_you_work_with_adequate_levels_of_support_staff?
0,2022-02-24T06:40:29Z,Registered Nurse,Progressive Care,5:1,5:1,Multiple times a week,NaN,Banner Baywood Medical Center,Chandler,Arizona,Absolutely powerless. I feel as if I’m letting...,No
1,2022-02-23T22:21:04Z,Registered Nurse,Emergency Room,4:1,4:1,Multiple times a week,NaN,Candler Hospital,Savannah,GA,NaN,Sometimes
2,2022-02-24T05:17:05Z,Registered Nurse,Maternal Care,3:1,4:1,Once every few months,NaN,The Christ Hospital Liberty Birthing Center,Liberty Township,Ohio,NaN,Yes
3,2022-02-23T21:57:16Z,Registered Nurse,Intensive Care Unit,2:1,4:1,Once a week,NaN,DRK Berlin Koepenick,Berlin,Germany,NaN,No
4,2022-02-24T01:59:37Z,Registered Nurse,Med-Surg Tele,5:1,6:1,Multiple times a week,NaN,Grand View Hospital,Sellersville,PA,NaN,No


In [11]:
facilities = (
    src.groupby(["city", "state", "facility_name"])
    .agg({"submission_time": "size"})
    .reset_index()
)

In [12]:
len(facilities)

318

#### Clean up facilities columns

In [13]:
facilities["state"] = facilities["state"].str.upper().str.replace(".", "", regex=False)
facilities["facility_name"] = (
    facilities["facility_name"].str.upper().str.replace(".", "", regex=False)
)

In [14]:
facilities[["city", "drop"]] = (
    facilities["city"].str.upper().str.split(",", expand=True)
)

In [15]:
facilities.drop(["drop", "submission_time"], axis=1, inplace=True)

In [16]:
facilities.head()

,city,state,facility_name
0,ABINGTON,PENNSYLVANIA,JEFFERSON ABINGTON HOSPITAL
1,ALBANY,NY,ALBANY MEDICAL CENTER
2,ALBEMARLE,NC,ATRIUM HEALTH
3,ALBEMARLE,NC,ATRIUM HEALTH STANLY
4,ALBEMARLE,NC,ATRIUM HEALTH


#### Clean up states

In [17]:
states.head()

,name,usps,ap
0,ALABAMA,AL,Ala.
1,ALASKA,AK,Alaska
2,ARIZONA,AZ,Ariz.
3,ARKANSAS,AR,Ark.
4,CALIFORNIA,CA,Calif.


#### Left merge to keep all facilities

In [18]:
merge = pd.merge(facilities, states, left_on="state", right_on="name", how="left")

#### If the merge locations didn't match, replace

In [19]:
merge["place"] = np.where(merge["usps"].isnull(), merge["state"], merge["usps"])

#### Lose places with long state names (Germany, British Columbia, etc) and columns we don't need

In [20]:
usa_df = merge[merge["place"].str.len() == 2].drop(
    ["state", "name", "usps", "ap"], axis=1
)

In [21]:
usa_df.head()

,city,facility_name,place
0,ABINGTON,JEFFERSON ABINGTON HOSPITAL,PA
1,ALBANY,ALBANY MEDICAL CENTER,NY
2,ALBEMARLE,ATRIUM HEALTH,NC
3,ALBEMARLE,ATRIUM HEALTH STANLY,NC
4,ALBEMARLE,ATRIUM HEALTH,NC


#### Another method for replacing values

In [22]:
states_map = {
    "AL": "Ala.",
    "AK": "Alaska",
    "AZ": "Ariz.",
    "AR": "Ark.",
    "CA": "Calif.",
    "CO": "Colo.",
    "CT": "Conn.",
    "DE": "Del.",
    "DC": "D.C.",
    "FL": "Fla.",
    "GA": "Ga.",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Ill.",
    "IN": "Ind.",
    "IA": "Iowa",
    "KS": "Kan.",
    "KY": "Ky.",
    "LA": "La.",
    "ME": "Maine",
    "MD": "Md.",
    "MA": "Mass.",
    "MI": "Mich.",
    "MN": "Minn.",
    "MS": "Miss.",
    "MO": "Mo.",
    "MT": "Mont.",
    "NE": "Neb.",
    "NV": "Nev.",
    "NH": "N.H.",
    "NJ": "N.J.",
    "NM": "N.M.",
    "NY": "N.Y.",
    "NC": "N.C.",
    "ND": "N.D.",
    "OH": "Ohio",
    "OK": "Okla.",
    "OR": "Ore.",
    "PA": "Pa.",
    "RI": "R.I.",
    "SC": "S.C.",
    "SD": "S.D.",
    "TN": "Tenn.",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vt.",
    "VA": "Va.",
    "WA": "Wash.",
    "WV": "W.Va.",
    "WI": "Wis.",
    "WY": "Wyo.",
}

In [23]:
usa_df["ap_name"] = usa_df["place"].map(states_map)

In [24]:
usa_df.head()

,city,facility_name,place,ap_name
0,ABINGTON,JEFFERSON ABINGTON HOSPITAL,PA,Pa.
1,ALBANY,ALBANY MEDICAL CENTER,NY,N.Y.
2,ALBEMARLE,ATRIUM HEALTH,NC,N.C.
3,ALBEMARLE,ATRIUM HEALTH STANLY,NC,N.C.
4,ALBEMARLE,ATRIUM HEALTH,NC,N.C.


In [25]:
usa_facilities = (
    usa_df.groupby(["city", "facility_name", "place", "ap_name"])
    .size()
    .reset_index(name="count")
)

In [26]:
usa_facilities.head()

,city,facility_name,place,ap_name,count
0,ABINGTON,JEFFERSON ABINGTON HOSPITAL,PA,Pa.,1
1,ALBANY,ALBANY MEDICAL CENTER,NY,N.Y.,1
2,ALBEMARLE,ATRIUM HEALTH,NC,N.C.,2
3,ALBEMARLE,ATRIUM HEALTH STANLY,NC,N.C.,1
4,ALBUQUERQUE,CHRISTUS ST VINCENT,NM,N.M.,2


In [27]:
len(usa_facilities)

297